In [2]:
conda install pytorch torchvision -c pytorch

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::anaconda==2019.07=py37_0
  - defaults/osx-64::numba==0.44.1=py37h6440ff4_0
done

## Package Plan ##

  environment location: /Users/gejiazhang/anaconda3

  added / updated specs:
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |           py37_0         2.8 MB
    ninja-1.9.0                |   py37h04f5b5a_0          90 KB
    pytorch-1.5.0              |          py3.7_0        34.7 MB  pytorch
    tbb-2020.0                 |       h04f5b5a_0         150 KB
    torchvision-0.6.0          |         py37_cpu         5.8 MB  pytorch
    ------------------------------------------------------------
                                           Total:        43.6 

In [1]:
from __future__ import print_function
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.6074, 0.6989, 0.7023],
        [0.4902, 0.0411, 0.2032],
        [0.8491, 0.2911, 0.2335],
        [0.9102, 0.6447, 0.3459],
        [0.3845, 0.3377, 0.0258]])


In [2]:
import torch
import torchvision

In [3]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [4]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./Documents/PythonWorkspace', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./Documents/PythonWorkspace', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

Extracting ./Documents/PythonWorkspace/MNIST/raw/train-images-idx3-ubyte.gz to ./Documents/PythonWorkspace/MNIST/raw


Extracting ./Documents/PythonWorkspace/MNIST/raw/train-labels-idx1-ubyte.gz to ./Documents/PythonWorkspace/MNIST/raw


Extracting ./Documents/PythonWorkspace/MNIST/raw/t10k-images-idx3-ubyte.gz to ./Documents/PythonWorkspace/MNIST/raw


Extracting ./Documents/PythonWorkspace/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./Documents/PythonWorkspace/MNIST/raw
Processing...


/Users/distiller/project/conda/conda-bld/pytorch_1587428061935/work/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Done!


In [5]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [6]:
example_data.shape

torch.Size([1000, 1, 28, 28])

In [7]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

<Figure size 640x480 with 6 Axes>

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [11]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [12]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [19]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), './Documents/PythonWorkspace/model.pth')
      torch.save(optimizer.state_dict(), './Documents/PythonWorkspace/optimizer.pth')

In [20]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [21]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/Users/gejiazhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. loss: 2.2919, Accuracy: 1090/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.294008
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.281380
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.296973
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.265300
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.252779
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.244723
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.258410
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.211463
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.162043
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.135211
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.087303
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.975637
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.960631
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.804835
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.675609
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.488627
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.545056
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.257703
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.299850
Train Epoch: 1 [12160

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.574812
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.475508
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.407826
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.453075
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.193857
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.622754
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.265453
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.420519
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.186983
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.336841
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.264929
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.216633
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.300108
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.161560
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.230581
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.234485
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.333287
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.563310
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.282829
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.307322
